In [1]:
import numpy as np
import pandas as pd
import pickle

import spacy
import nltk
import en_core_web_sm
from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

D:\anaconda3\envs\NLP\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
#!python -m spacy download en_core_web_sm
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [3]:
with open('data/tweet_data.pkl', 'rb') as f:
    combine_df  = pickle.load(f)

In [4]:
combine_df .head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [5]:
%%time
nlp = en_core_web_sm.load()
combine_df['nlp'] = combine_df.clean_tweet.progress_apply(lambda x:  nlp(x))

100%|███████████████████████████████████████████████████████████████████████████| 49159/49159 [04:16<00:00, 191.36it/s]

Wall time: 4min 17s


In [6]:
nlp.max_length = combine_df['nlp'].shape[0]

In [7]:
labels = []
for doc in combine_df['nlp'].tolist():
    for e in doc.ents:
        labels.append((e.label_, e.text))

In [9]:
ner_df = pd.DataFrame(labels, columns=['ner', 'word'])

In [10]:
ner_df.ner.value_counts().head(20)

DATE           11515
PERSON          7999
ORG             5563
GPE             4572
TIME            2033
NORP            1451
CARDINAL        1091
ORDINAL          648
FAC              302
LOC              217
EVENT            150
PRODUCT           60
LANGUAGE          42
QUANTITY          34
WORK_OF_ART       27
LAW               22
MONEY              8
PERCENT            3
Name: ner, dtype: int64

In [11]:
ner_df.loc[ner_df['ner'] == 'PERSON'].word.value_counts().head(20)

hea                                                        119
hu                                                          87
blur sun                                                    55
bihday                                                      47
feminismiscancer feminismisterrorism feminismmuktbharat     40
christina grimmie                                           38
hillary                                                     37
sikh temple                                                 28
tgif ff                                                     26
don                                                         24
detoxdiet altwaystoheal                                     21
clinton                                                     21
ripchristina                                                20
jo cox                                                      20
carl paladino                                               19
regrann                                                

In [12]:
ner_df.loc[ner_df['ner'] == 'ORG'].word.value_counts().head(20)

bong bing           107
app                  84
gop                  77
islam                58
house                46
social analytics     40
nba                  39
usa                  35
amazon               32
sta                  31
sma                  30
cnn                  28
euro                 27
fed                  25
ios                  23
eu                   22
fbi                  21
sun                  20
bogota colombia      20
congress             19
Name: word, dtype: int64

### Задание 2.

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [14]:
string_tweet = combine_df.tweet.to_string()

In [15]:
string_tweet_tokenize = nltk.word_tokenize(string_tweet)

In [16]:
string_tweet_tags = nltk.pos_tag(string_tweet_tokenize)

In [17]:
string_tweet_chunk = nltk.ne_chunk (string_tweet_tags)

In [22]:
{(' '.join(c[0] for c in chunk), chunk.label() ) \
 for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(string_tweet))) if hasattr(chunk, 'label') }

{('___', 'ORGANIZATION'),
 ('______________', 'ORGANIZATION'),
 ('_______________________', 'ORGANIZATION'),
 ('atÂ', 'ORGANIZATION'),
 ('babalargÃ¼nÃ¼', 'ORGANIZATION'),
 ('bearishÂ', 'ORGANIZATION'),
 ('braggingÂ', 'ORGANIZATION'),
 ('brÃ¼ssel', 'ORGANIZATION'),
 ('eatmorecrÃ', 'ORGANIZATION'),
 ('en-ger-land', 'GPE'),
 ('euro2016Â', 'ORGANIZATION'),
 ('euro2016Â Â', 'ORGANIZATION'),
 ('felicitÃ', 'ORGANIZATION'),
 ('firstÂ', 'ORGANIZATION'),
 ('foiinÃ', 'ORGANIZATION'),
 ('fridayÂ', 'ORGANIZATION'),
 ('fÃªte', 'ORGANIZATION'),
 ('gymð\x9f\x92ª', 'ORGANIZATION'),
 ('gÃ¼mbet', 'ORGANIZATION'),
 ('ilustraciÃ³n', 'ORGANIZATION'),
 ('inc.', 'ORGANIZATION'),
 ('l.a.', 'GPE'),
 ('marschfÃ¼rjesus', 'ORGANIZATION'),
 ('mr.', 'PERSON'),
 ('mr. paris', 'PERSON'),
 ('nycÂ', 'ORGANIZATION'),
 ('oilÂ', 'ORGANIZATION'),
 ('peace/sad', 'ORGANIZATION'),
 ('st.peter', 'ORGANIZATION'),
 ('sunflowerÂ', 'ORGANIZATION'),
 ('turÂ', 'ORGANIZATION'),
 ('u.s', 'GPE'),
 ('u.s.', 'GPE'),
 ('vysoÄ\x8dina', 'ORG